In [1]:
import pandas as pd
from config import CONFIG

# PART 1 : Demonstration of my understanding of the ETL pipeline

I have tried to focus on the process and produce some working code. The python files are numbered to indicate the sequence in which they should be executed.

I have used only the "product_invoices" table as an example for the Database part as a demo. The field definitions for the other tables can be made using the same technique.

The schema for "product_invoices" is defined in the config.py file. It generates the necessary SQL to insert the data what has been collated from the json files. SQLite is used for simplicity. 

It is assumed that the data zip file has been unzipped in the current directory under the "data" folder.

The generated CSV files are available in the csv folder. Processed json files are archived in the processed folder of each subfolder.

Please take a look at config.py for updating simple config settings. the library files my_utils.py and my_db_utils.py contains all the functions I have written.

To schedule batch processing, we can use Prefect (https://www.prefect.io) or Airflow or a chron job to run these utilities as shown below. The code in the library can be reused with minimal change in a full fledged python script if needed.

A developer friendly screen logging utility called ice-cream or ic has been used to show a self explanatory description of the flow.

In [2]:
# SHIPPING_DATA.sqlite3 will be generated in the root folder
!./01-create-db.py

ic| sql_create: ('CREATE TABLE IF NOT EXISTS product_invoices (transaction_id TEXT, amount '
                 'REAL, user_invoice_date TEXT);')


In [3]:
# Generate csv files from the given json files
!./02-json-to-csv.py

ic| f'added {filename}': ('added '
                          './data/product_invoices/part-00000-559320dc-cc27-4b64-a490-c8449fa63c0d-c000.json')
ic| f'added {filename}': ('added '
                          './data/product_invoices/part-00001-559320dc-cc27-4b64-a490-c8449fa63c0d-c000.json')
ic| f'added {filename}': ('added '
                          './data/product_invoices/part-00002-559320dc-cc27-4b64-a490-c8449fa63c0d-c000.json')
ic| f'added {filename}': ('added '
                          './data/product_invoices/part-00003-559320dc-cc27-4b64-a490-c8449fa63c0d-c000.json')
ic| f'added {filename}': ('added '
                          './data/product_invoices/part-00004-559320dc-cc27-4b64-a490-c8449fa63c0d-c000.json')
ic| f'added {filename}': ('added '
                          './data/product_invoices/part-00005-559320dc-cc27-4b64-a490-c8449fa63c0d-c000.json')
ic| f'added {filename}': ('added '
                          './data/product_invoices/part-00006-559320dc-cc27-4b64-a490-c84

In [4]:
# Insert the data into the SQLite DB
!./03-csv-to-sqlite.py

ic| 'Rows available:'
ic| len(rows): 554038
ic| sql_insert: 'INSERT INTO product_invoices VALUES (?, ?, ?);'
ic| 'insert completed'
ic| 'imported ' + csv_file: 'imported ./csv/product_invoices.csv'


In [5]:
# Check if it went into the DB
!./04-database-view.py

ic| sql_select: 'SELECT * FROM product_invoices LIMIT 100;'
ic| 'query executed'
ic| data: [('221271390', 3.15, '2018-12-20'),
           ('153588634', 3.15, '2018-08-24'),
           ('270671460', 2.4, '2019-03-01'),
           ('158409145', 2.73, '2018-09-04'),
           ('178911135', 2.73, '2018-10-18'),
           ('76921351', 2.4, '2017-12-11'),
           ('172136340', 3.15, '2018-09-29'),
           ('79894251', 3.15, '2017-12-18'),
           ('56800771', 3.15, '2017-09-07'),
           ('57842355', 2.73, '2017-09-02'),
           ('155300445', 2.73, '2018-10-02'),
           ('208223642', 2.73, '2018-12-02'),
           ('160568491', 2.73, '2018-09-10'),
           ('129937277', 2.4, '2018-06-09'),
           ('153761864', 2.4, '2018-08-23'),
           ('116139271', 3.15, '2018-05-04'),
           ('119494618', 2.4, '2018-05-12'),
           ('50782102', 2.4, '2017-06-26'),
           ('163367320', 3.15, '2018-09-13'),
           ('78401161', 2.4, '2017-12-21'),
           (

# PART 2 : Simple analysis of the given data

I have done some elementary analysis, as seen below. More time would be needed to make a more detailed one. I would have liked to do some transformation on the product_package_types table using map() as there are several duplicates. 

In [6]:
csv_dir = CONFIG['csv_dir']

df_product_invoices = pd.read_csv(f'{csv_dir}/product_invoices.csv', index_col='transaction_id')
df_product_package_types = pd.read_csv(f'{csv_dir}/product_package_types.csv', index_col='id')
df_product_shipments = pd.read_csv(f'{csv_dir}/product_shipments.csv', index_col='transaction_id')
df_provider_invoices = pd.read_csv(f'{csv_dir}/provider_invoices.csv', index_col='tracking_code')
df_provider_prices = pd.read_csv(f'{csv_dir}/provider_prices.csv', index_col=['from_country', 'to_country'])

In [7]:
df_product_invoices

,amount,user_invoice_date
transaction_id,,
221271390,3.150000,2018-12-20
153588634,3.150000,2018-08-24
270671460,2.400000,2019-03-01
158409145,2.730000,2018-09-04
178911135,2.730000,2018-10-18
...,...,...
131206431,2.400000,2018-06-13
252787333,2.400000,2019-02-08
250082599,2.400000,2019-01-30


In [8]:
df_product_package_types.value_counts()

description                               
Max. 20 kg, 120 x 60 x 60 cm                  36
Max. 20 kg, 100 x 50 x 50 cm                  24
5kg                                           14
2kg                                            7
0.5kg                                          6
1kg                                            6
10kg                                           5
20kg                                           5
Max. 10 kg, 100 × 50 × 50 cm                   4
30x30x30cm 0-2kg                               3
Max. 2 kg, Largo + Alto + Ancho ≤ 210 cm       2
Max. 5 kg, Largo + Alto + Ancho ≤ 210 cm       2
Max. 20 kg, Largo + Alto + Ancho ≤ 210 cm      2
Max. 1 kg, Largo + Alto + Ancho ≤ 210 cm       2
Max. 10 kg, Largo + Alto + Ancho ≤ 210 cm      2
Max. 0.5 kg, Largo + Alto + Ancho ≤ 210 cm     2
Épaiss. max 3cm 0.25kg                         2
Max. 2 kg, 38 × 26,5 × 3,2 cm                  1
0.75kg                                         1
dtype: int64

In [9]:
df_product_shipments

,tracking_code,from_country,to_country,package_type_id,shipping_label_created,buyer_id,seller_id
transaction_id,,,,,,,
182260673,30611711,FR,FR,11,2018-10-13 10:38:35,16471055,18098417
203716496,33192017,FR,FR,10,2018-11-14 18:43:26,511189,16281325
203394963,33286778,FR,FR,11,2018-11-15 21:39:24,13351339,15644819
161640430,28134416,FR,FR,11,2018-09-06 09:44:07,18319157,855863
174990551,29892135,FR,FR,10,2018-10-04 08:37:26,16998722,1691012
...,...,...,...,...,...,...,...
200390728,33409354,FR,FR,12,2018-11-17 18:21:51,19699917,16662926
265068264,40949324,FR,FR,10,2019-02-17 19:14:26,11879770,15749068
210053751,33929048,FR,FR,10,2018-11-24 08:56:13,11880641,15285615


In [10]:
df_provider_invoices

,from_country,to_country,weight_measured,amount
tracking_code,,,,
36075318,FR,FR,0.0,2.40
37899329,FR,FR,870.0,2.73
36344250,FR,FR,280.0,2.40
38425854,FR,FR,160.0,2.40
15587085,FR,FR,630.0,3.28
...,...,...,...,...
39937773,FR,FR,540.0,2.73
39588954,FR,FR,1200.0,3.15
35152712,FR,FR,540.0,2.40


In [11]:
df_provider_prices

price actual_package_size
from_country to_country                           
FR           FR           2.40              0.5 kg
             FR           2.73                1 kg
             FR           3.15                2 kg
             FR           3.54                3 kg
             FR           4.27                5 kg
...                        ...                 ...
ES           LU           6.77                5 kg
             LU           8.96                7 kg
             LU          11.75               10 kg
             LU          15.58               15 kg
             LU          25.01               30 kg

[144 rows x 2 columns]

In [12]:
df_provider_prices['actual_package_size_grams'] = df_provider_prices['actual_package_size'].str.replace(' kg', '').astype(float)*1000
df_provider_prices['actual_package_size_grams'] = df_provider_prices['actual_package_size_grams'].astype(int)
df_provider_prices

price actual_package_size  actual_package_size_grams
from_country to_country                                                      
FR           FR           2.40              0.5 kg                        500
             FR           2.73                1 kg                       1000
             FR           3.15                2 kg                       2000
             FR           3.54                3 kg                       3000
             FR           4.27                5 kg                       5000
...                        ...                 ...                        ...
ES           LU           6.77                5 kg                       5000
             LU           8.96                7 kg                       7000
             LU          11.75               10 kg                      10000
             LU          15.58               15 kg                      15000
             LU          25.01               30 kg                      30000

[144 rows x 3 columns]

In [13]:
df_provider_prices.dtypes

price                        float64
actual_package_size           object
actual_package_size_grams      int64
dtype: object

In [14]:
df1 = pd.merge(df_product_shipments, df_product_invoices, on='transaction_id', how='inner')
df1['copy_transaction_id'] = df1.index
df1.rename(columns={'copy_transaction_id':'transaction_id'}, inplace=True)
df1

,tracking_code,from_country,to_country,package_type_id,shipping_label_created,buyer_id,seller_id,amount,user_invoice_date,transaction_id
transaction_id,,,,,,,,,,
182260673,30611711,FR,FR,11,2018-10-13 10:38:35,16471055,18098417,2.73,2018-10-22,182260673
203716496,33192017,FR,FR,10,2018-11-14 18:43:26,511189,16281325,2.40,2018-11-24,203716496
203394963,33286778,FR,FR,11,2018-11-15 21:39:24,13351339,15644819,2.73,2018-12-02,203394963
161640430,28134416,FR,FR,11,2018-09-06 09:44:07,18319157,855863,2.73,2018-09-17,161640430
174990551,29892135,FR,FR,10,2018-10-04 08:37:26,16998722,1691012,2.40,2018-10-07,174990551
...,...,...,...,...,...,...,...,...,...,...
200390728,33409354,FR,FR,12,2018-11-17 18:21:51,19699917,16662926,3.15,2018-11-25,200390728
265068264,40949324,FR,FR,10,2019-02-17 19:14:26,11879770,15749068,2.40,2019-02-22,265068264
210053751,33929048,FR,FR,10,2018-11-24 08:56:13,11880641,15285615,2.40,2018-11-30,210053751


In [15]:
df = pd.merge(df_provider_invoices, df1, on='tracking_code', how='inner')
df = df.set_index('tracking_code')
df

,from_country_x,to_country_x,weight_measured,amount_x,from_country_y,to_country_y,package_type_id,shipping_label_created,buyer_id,seller_id,amount_y,user_invoice_date,transaction_id
tracking_code,,,,,,,,,,,,,
36075318,FR,FR,0.0,2.40,FR,FR,10,2018-12-19 12:04:48,13296907,19062706,2.400000,2018-12-27,225783345
37899329,FR,FR,870.0,2.73,FR,FR,11,2019-01-14 16:47:31,19275119,20812041,2.730000,2019-01-20,240704671
36344250,FR,FR,280.0,2.40,FR,FR,10,2018-12-25 11:42:25,12954072,12255702,2.400000,2019-01-04,227924168
38425854,FR,FR,160.0,2.40,FR,FR,11,2019-01-21 07:06:18,20316153,15129902,2.730000,2019-01-29,238469231
15587085,FR,FR,630.0,3.28,FR,FR,12,2017-05-11 13:36:20,11989238,1612758,3.150000,2017-05-18,45988459
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39937773,FR,FR,540.0,2.73,FR,FR,11,2019-02-06 11:25:27,11470912,14463363,2.733333,2019-02-12,252595290
39588954,FR,FR,1200.0,3.15,FR,FR,12,2019-02-03 09:00:27,13221390,14699732,3.150000,2019-02-08,254298186
35152712,FR,FR,540.0,2.40,FR,FR,10,2018-12-08 12:43:53,16001541,880843,2.400000,2018-12-14,219250271


In [16]:
df['amount_x'].sum()

1465138.1999999997

In [17]:
df['amount_diff']=df['amount_x'] - df['amount_y']

In [18]:
df_plus = df[df['amount_diff'] > 0.0]
df_plus[['amount_x', 'amount_y', 'amount_diff']]

,amount_x,amount_y,amount_diff
tracking_code,,,
15587085,3.28,3.15,0.13
17995763,3.15,2.73,0.42
38717318,3.15,2.73,0.42
18837465,3.15,2.73,0.42
15636656,3.28,2.73,0.55
...,...,...,...
13673492,2.88,2.40,0.48
14924672,3.28,2.73,0.55
13804124,3.28,2.73,0.55


In [19]:
df_plus['amount_diff'].sum()

31825.113310999997

In [20]:
df_plus['amount_diff'].count()

68266

In [21]:
df_plus['amount_diff'].mean()

0.46619273593003835

In [22]:
df_plus['amount_diff'].sum()/df['amount_x'].sum()*100

2.1721577739901945

In [23]:
df_minus = df[df['amount_diff'] < 0.0]
df_minus[['amount_x', 'amount_y', 'amount_diff']]

,amount_x,amount_y,amount_diff
tracking_code,,,
38425854,2.40,2.730000,-0.330000
24615486,2.40,3.150000,-0.750000
39849261,2.73,2.733333,-0.003333
17615666,2.73,3.150000,-0.420000
17590986,2.40,2.730000,-0.330000
...,...,...,...
31351033,2.40,2.730000,-0.330000
17089069,2.40,2.730000,-0.330000
29319006,2.73,3.150000,-0.420000


In [24]:
df_minus['amount_diff'].sum()

-37542.552563000005

In [25]:
df_minus['amount_diff'].count()

119336

In [26]:
df_minus['amount_diff'].mean()


-0.31459536571529134

In [27]:
df['amount_diff'].sum()

-5717.439252000006

# Conclusion:

The number above suggests that there is a difference between the amount of shipping paid and collected. The snapshot of data available shows that overall 5717 more was received than paid out. However there have been several buyers who have paid more and several who have paid less. On average, +46 cents vs -31 cents respectively.

My conclusion is that the sellers do not have proper equipment to weigh the parcels and this leads to them guesstimating the weights thus leading to an inaccurate shipping cost being paid by the buyers.

Some differences could also be due to rounding errors.

Since buyers who pay less than they were supposed to won't really mind, more attention needs to be paid to the ones who overpaid. This could lead to resentment and could hurt the brand equity of the Company. However in percentage terms, it is a relatively small amount (2.17% of the total shipping collected). It would be the decision of the stakeholders about what the tolerance level should be.

A simple solution would be to refund the difference to the buyers after a reconciliation of the actual charge from the provider. Also, helping the sellers to come to a more accurate weight would be something that could be considered.

# Request for more data

It would be very interesting to get the weight of the package that was entered by the seller to generate the Shipping Label.

It would also be useful to clean up the provider_prices and product_table_types data in some way.

Since the available data is only a snapshot, it would be helpful to run this analysis on a more comprehensive dataset to get a better feel for the actual percentage of error.

Finally, I would ask for the returns and cancellations data to see if it is being handled correctly.

Thank you for reading!